# LLM工具說明

## llm_tools模組說明
### 參數設定
- 查看configs/llm.ini
    - **DEFAULT**: 預設的參數，包含temperature、max_tokens、top_p、frequency_penalty、presence_penalty等
    - **Azure OpenAI系列**: 設定azure_api_base、azure_api_key、azure_api_version，打了會花錢，請謹慎使用
    - **地端LLM/Embedding Model系列**: 設定local_api_key、local_base_url，目前統一接口8887，若有實驗新模型可自行新增
  
### 導入方法
- 第一步：將llm_tools資料夾放在跟操作程式碼相同路徑
- 第二步：在python中導入以下模組
    - from llm_tools.llm_chat import LLMChat
    - from llm_tools.embedding_model import EmbeddingModel

### 函數內容
- llm_chat.py 重要參數與功能
    - params (dictionary): 可控制參數
    - response_format (string): 可設定"json_object"，讓LLM返回json物件，但是prompt內要有json的說明
    - stream (bool): 設定為True可使用流式輸出
    - history (list): OpenAI接口回傳的history，格式為[{"role": "system", "content": system_prompt}, {"role": "user", "content": query}. {"role": "system", "content": completion}]
- embedding_model.py 重要參數與功能
    - 單句轉Embedding: 輸出為array
    - 多句(list)轉Embedding: 會集合成batch方式轉譯，跑批的時候速度較快，輸出為array

## LLM單句對答
- 可測stream、json回答等
- response會把stream的回答合併起來，所以用stream模式如果print(response)會看到兩次回答

In [1]:
from llm_tools.llm_chat import LLMChat

# Test LLM
# llmchat = LLMChat(model="gpt-4o")
llmchat = LLMChat(model="Qwen1.5-14B-Chat")
# llmchat = LLMChat(model="Qwen2-7B-Instruct")

params = {
    "temperature": 0.8,
    "max_tokens": 1000,
    "top_p": 1,
    "frequency_penalty": 1.4,
    "presence_penalty": 0
}
try:
    system = """
    任務：解讀主題與內容
    請從下面的文字中解讀該段落的語意主題與內容，應該詳細閱讀資訊，並提取其語意主題與描述內容。
    """
    query = """
    咱們的護國神山，台積電在今年四月的北美技術論壇上，發表埃米級的 A16 製程，預計於 2026 年開始量產，引發各大科技龍頭的瘋搶。
    台積的 A16 製程更引入 SPR 系統，不只減少了 IR 降壓，相較於過去的 N2P 製程，能在相同性能下，減少最多 20% 的功耗，同時提升近 10% 的晶片密度。
    OpenAI 也希望得到 A16 製程的協助，打造屬於自家的 AI 晶片，用以強化旗下的 GPT 語言模型，以及影像生成模型 Sora。
    """
    history = None
    # response, history = llmchat.chat(query=query, system=system, params=params, response_format="json_object", stream=True) # response規定json
    response, history = llmchat.chat(query=query, system=system, stream=True)
    print()
    print(response)
except ValueError as e:
    print(e)

語意主題：台積電 A16 製程及其影響

內容描述：
1. 主題：台積電在北美技術論壇上展示了埃米級（A16）製程，這是一個重要的科技發展里程碑，預計於 2026 年開始量產。
2. 具體細節：A16 製程引發了各大科技公司的關注和競爭，特別是 OpenAI 等公司對其有興趣。
3. 技術特點：A16 製程引入了 SPR 系統，相較於之前的 N2P 製程，在保持相同性能的同時能降低最多 20% 的功耗，并提升近 10% 的晶片密度。這顯示出更高的能源效率和更小的芯片尺寸。
4. 目標應用：OpenAI 想要利用 A16 製程來開發自家的 AI 晶片，以增強他們的 GPT語言模型和影像生成模型 Sora的能力。

總結來說，該段落主要討論的是台積電新製程 A16 的技術進步、其優勢以及對人工智能領域如 OpenAI 所帶來的可能性。
語意主題：台積電 A16 製程及其影響

內容描述：
1. 主題：台積電在北美技術論壇上展示了埃米級（A16）製程，這是一個重要的科技發展里程碑，預計於 2026 年開始量產。
2. 具體細節：A16 製程引發了各大科技公司的關注和競爭，特別是 OpenAI 等公司對其有興趣。
3. 技術特點：A16 製程引入了 SPR 系統，相較於之前的 N2P 製程，在保持相同性能的同時能降低最多 20% 的功耗，并提升近 10% 的晶片密度。這顯示出更高的能源效率和更小的芯片尺寸。
4. 目標應用：OpenAI 想要利用 A16 製程來開發自家的 AI 晶片，以增強他們的 GPT語言模型和影像生成模型 Sora的能力。

總結來說，該段落主要討論的是台積電新製程 A16 的技術進步、其優勢以及對人工智能領域如 OpenAI 所帶來的可能性。


## 測試LLM多輪問答
- 用while迴圈來做多輪問答看看唄
- 輸入quit結束問答

In [2]:
from llm_tools.llm_chat import LLMChat

# Test LLM Chatbot
llmchat = LLMChat(model="Qwen2-7B-Instruct")
history=None 
is_stream=True
while True:
    input_text = input("請輸入你的問題: ")  
    if input_text.lower() == "quit":  
        break
    response, history = llmchat.chat(query=input_text, history=history, stream=is_stream)
    if is_stream==True:
        print()
    else:
        print(response)

請輸入你的問題:  第一次問答


當然，我很樂意回答您的問題。請問您有什麼想要了解的呢？


請輸入你的問題:  第二次問答


非常好，如果您有任何问题，无论是关于学术、科技、生活常识还是其他任何主题，请随时向我提问。我会尽力提供准确和有用的信息。那么请问您有什么想要了解的内容呢？


請輸入你的問題:  第三次問答


当然，很高兴再次为您服务。请问您有什么问题或需要讨论的主题吗？无论是关于学习、工作、生活建议，还是任何其他领域的问题，我都将尽力提供帮助。


請輸入你的問題:  我問了幾次?


您已经进行了三次问答回合。每次问答回合包括一次初始询问和两次后续回应，因此总共是三次问答回合。


請輸入你的問題:  quit


## Embedding測試
- 單句轉Embedding與多句轉Embedding

In [4]:
import numpy as np
from llm_tools.embedding_model import EmbeddingModel

# Test EmbeddingModel
embmodel = EmbeddingModel(embedding_model="m3e-base")
query = "The food was delicious and the waiter was friendly."
query_embedding = np.array(embmodel.embed_query(query))
print(len(query_embedding))
documents = ["The food was delicious and the waiter was friendly.",
             "The service was slow and the food was not very good."]
document_embeddings = np.array(embmodel.embed_documents(documents))
print(document_embeddings.shape)

768
(2, 768)
